In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

In [ ]:
data = pd.read_excel("Датасет.xlsx")

In [ ]:
df = pd.DataFrame()
df.loc[:, 'responsibilities'] = data['responsibilities(Должностные обязанности)']
df.loc[:, 'requirements'] = data['requirements(Требования к соискателю)']
df.loc[:, 'terms'] = data['terms(Условия)']
df.loc[:, 'notes'] = data['notes(Примечания)']

for i in range(df.shape[0]):
  df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['requirements'][i]), "")
  df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['terms'][i]), "")
  df['responsibilities'][i] = df['responsibilities'][i].replace(str(df['notes'][i]), "")

In [ ]:
df[:10]

,responsibilities,requirements,terms,notes
0,,NaN,NaN,Работа на строительных площадках. Очистных соо...
1,Гнутье арматурной стали на механических станка...,NaN,NaN,NaN
2,Вязка арматуры.,NaN,NaN,NaN
3,. Выполнение работ по сборке и вязке арматурны...,Выполнение работ по гнутью и резке арматурной ...,NaN,\N
4,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Официальное трудоустройство; - ЗП в срок и без...
5,Требуется АРМАТУРЩИК на крупнейший завод по пр...,"Опыт работы от 1 года , наличие удостоверения ...",Желание работать и зарабатывать Условия: Рабо...,Ставка за смену 3080 руб при графике 6/1 по 11...
6,Почепта Дальневосточной железной дороги Мост ...,опыт в строительстве приветствуется -работа в ...,продолжительность вахты 60/30 (продление вахты...,Помощь в прохождение медицинского осмотра; - В...
7,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; - работа ...,продолжительность вахты 60/30 (продление вахты...,Официальное трудоустройство; - ЗП в срок и бе...
8,Подготовка участка к укладке дорожного покрыти...,NaN,NaN,NaN
9,Приготовление асфальтовых смесей на передвижны...,NaN,NaN,NaN


In [ ]:
df_1 = pd.DataFrame(columns = ["Text", "Class", "Preproccessed"])


for x in df['responsibilities']:
  df_1 = df_1.append({"Text": x, "Class": "responsibilities"}, ignore_index=True)

for x in df['requirements']:
  df_1 = df_1.append({"Text": x, "Class": "requirements"}, ignore_index=True)

for x in df['terms']:
  df_1 = df_1.append({"Text": x, "Class": "terms"}, ignore_index=True)

for x in df['notes']:
  df_1 = df_1.append({"Text": x, "Class": "notes"}, ignore_index=True)

df_1 = df_1.dropna(subset=['Text'])


In [ ]:
df_1 = df_1.reset_index()
df_1 = df_1.drop(['index'], axis=1)

In [ ]:
def remove_stickers(text):
  if text != text:
    return np.nan
  # Паттерн для поиска стикеров в формате [стикер]
  pattern = r"[^\w.!?,:;$\s/\"\']*"
  cleaned_text = re.sub(pattern, "", str(text))
  return cleaned_text

for i in range(df_1.shape[0]):
  df_1['Text'][i] = remove_stickers(str(df_1['Text'][i]))

In [ ]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

import nltk
nltk.download('stopwords')
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem()

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

In [ ]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))))
df_1['Preproccessed'] = list(map(preproccessing, df_1['Text']))

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [ ]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [ ]:
from nltk import word_tokenize
nltk.download('punkt')

In [ ]:
stemmed_texts_list = []
for text in tqdm(df_1['Preproccessed']):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_1['text_stem'] = stemmed_texts_list

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
sw_texts_list = []
for text in tqdm(df_1['Preproccessed']):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_1['text_sw'] = sw_texts_list

In [110]:
lemm_texts_list = []
for text in tqdm(df_1['text_sw']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)

df_1['text_lemm'] = lemm_texts_list

  0%|          | 0/2936 [00:00<?, ?it/s]

In [111]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [166]:
df_1[:10]

,Text,Class,Preproccessed,text_stem,text_sw,text_lemm
0,"Вахта 60/30 Проезд, проживание, питание. Пря...",responsibilities,вахта проезд проживание питание прямой работод...,вахт проезд проживан питан прям работодател мо...,вахта проезд проживание питание прямой работод...,вахта проезд проживание питание прямая работод...
1,Медкомиссия за счет работодателя Выдача спец...,terms,медкомиссия за счет работодателя выдача спецод...,медкомисс счет работодател выдач спецодежд раб...,медкомиссия счет работодателя выдача спецодежд...,медкомиссия счет работодатель выдача спецодежд...
2,quot;РЕГРОquot; является промышленностроительн...,responsibilities,quot регроquot является промышленностроительны...,quot регроqуот явля промышленностроительн пред...,quot регроquot является промышленностроительны...,quot регроquot являться промышленностроительны...
3,оплата сдельная,terms,оплата сдельная,оплат сдельн,оплата сдельная,оплата сдельный \n
4,Строительство мостового перехода в г.Новосибир...,responsibilities,строительство мостового перехода в г новосибир...,строительств мостов переход г новосибирск треб...,строительство мостового перехода г новосибирск...,строительство мостовый переход г новосибирск т...
5,Среднее общее; среднее специальное образование...,requirements,среднее общее среднее специальное образование ...,средн общ средн специальн образован общ стаж р...,среднее общее среднее специальное образование ...,средний общий средний специальный образование ...
6,Мы гарантируем: Постоянный поток заказов; Зад...,responsibilities,мы гарантируем постоянный поток заказов задачи...,гарантиру постоя поток заказ задач работ получ...,гарантируем постоянный поток заказов задачи ра...,гарантировать постоянный поток заказ задача ра...
7,"управление краном 25 тонн, или управление кран...",responsibilities,управление краном тонн или управление краном с...,управлен кран тон управлен кран полапульт опла...,управление краном тонн управление краном полап...,управление кран тонна управление кран полапуль...
8,Готовность к физической работе; Можно без опы...,requirements,готовность к физической работе можно без опыта...,готовн физическ работ опыт умен пользова элект...,готовность физической работе опыта умение поль...,готовность физический работа опыт умение польз...
9,ЗВОНИТЕ ПРЯМО СЕЙЧАС Также готовы рассмотреть...,notes,звоните прямо сейчас также готовы рассмотреть ...,звон прям такж готов рассмотрет сотрудник опыт...,звоните прямо также готовы рассмотреть сотрудн...,звонить прямо также готовый рассматривать сотр...


In [167]:
X = df_1['text_lemm']
#X = df_1['text_sw']
y = df_1['Class']

In [168]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# Naive Bayes Classifier

In [169]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [170]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [171]:
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [172]:
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [175]:
y_pred[0]

'requirements'

In [178]:
from sklearn.metrics import accuracy_score

classes = df_1['Class'].unique()

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=classes))



accuracy 0.8479001135073779
                  precision    recall  f1-score   support

responsibilities       1.00      0.39      0.56        67
           terms       0.95      0.84      0.90       250
    requirements       0.77      0.89      0.82       293
           notes       0.85      0.92      0.88       271

        accuracy                           0.85       881
       macro avg       0.89      0.76      0.79       881
    weighted avg       0.86      0.85      0.84       881



In [250]:
text = ""
text = remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower())))
text = remove_stop_words(text)
text = lemmatize_text(text)

pred = nb.predict([text])
pred

array(['terms'], dtype='<U16')

# Linear Support Vector Machine

In [187]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [188]:
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [190]:
y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

accuracy 0.8819523269012486
                  precision    recall  f1-score   support

responsibilities       0.97      0.46      0.63        67
           terms       0.91      0.90      0.91       250
    requirements       0.84      0.92      0.88       293
           notes       0.90      0.92      0.91       271

        accuracy                           0.88       881
       macro avg       0.90      0.80      0.83       881
    weighted avg       0.89      0.88      0.88       881



In [248]:
text = "Авансирование дважды в месяц по 15 000 рублей, 15 и 30 числа"
text = remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower())))
text = remove_stop_words(text)
text = lemmatize_text(text)

pred = sgd.predict([text])
pred

array(['terms'], dtype='<U16')

# Logistic Regression

In [238]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=100)),
               ])

In [239]:
logreg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [240]:
y_pred = logreg.predict(X_test)

In [241]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

accuracy 0.8842224744608399
                  precision    recall  f1-score   support

responsibilities       0.82      0.60      0.69        67
           terms       0.95      0.85      0.89       250
    requirements       0.84      0.94      0.89       293
           notes       0.89      0.93      0.91       271

        accuracy                           0.88       881
       macro avg       0.88      0.83      0.85       881
    weighted avg       0.89      0.88      0.88       881



In [260]:
text = "Работа Маляр в Москве ПО 8 ЧАСОВ. Вакансия Маляр в строительстве на чистовую отделку больших объемов! Работа!  Открыт прием на несколько строительных объектов по Москве на выбор!  Большие ставки, готовы давать этажи бригадам!"
text = remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower())))
text = remove_stop_words(text)
text = lemmatize_text(text)

pred = logreg.predict([text])
pred

array(['responsibilities'], dtype=object)